# Space Mice Genes
## Heat Propagation and Clustering Package


----------------------

Author: Mikayla Webster (13webstermj@gmail.com)

Date: 6th April, 2018

----------------------

<a id='toc'></a>
## Table of Contents
1. [Background](#background)
2. [Import packages](#import)
3. [Load Networks](#load)
4. [Run Heat Propagation](#heat)
5. [Clustering](#cluster)

## Background
<a id='background'></a>

## Import packages
<a id='import'></a>

In [1]:
import sys
code_path = '../../network_bio_toolkit'
sys.path.append(code_path)

import create_graph
reload(create_graph)

import heat_and_cluster
reload(heat_and_cluster)

import visJS2jupyter.visualizations as visualizations

import networkx as nx

## Load Networks
<a id='load'></a>

1. Load DEG file 
2. Load STRING background network

In [2]:
# differentially expressed genes
DEG_filename = "../../DEG_databases/DE_CoeffspaceFlight - groundControl_glds48_20180312.csv"  
DEG_list, DEG_to_pvalue, DEG_to_updown = create_graph.create_DEG_list(DEG_filename, 
                                                                      p_value_filter=0.1,
                                                                      gene_type='symbol',
                                                                      sep = ',')

print "Number of DEG's: " + str(len(DEG_list))

Number of DEG's: 394


In [3]:
# background network
filename = "../../background_networks/10090.protein.actions.v10.5.txt"
G_DEG, DG_universe = create_graph.load_STRING_to_digraph(filename, DEG_list, confidence_filter = 400, gene_type = 'symbol', species = 'mouse')

print "\nNumber of interactions: " + str(len(list(G_DEG.edges())))

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-7303...done.
Finished.
57 input query terms found dup hits:
	[(u'ENSMUSP00000042658', 2), (u'ENSMUSP00000126737', 2), (u'ENSMUSP00000051968', 3), (u'ENSMUSP00000
154 input query terms found no hit:
	[u'ENSMUSP00000036849', u'ENSMUSP00000097354', u'ENSMUSP00000097358', u'ENSMUSP00000137522', u'ENSMU
Pass "returnall=True" to return complete lists of duplicate or missing query terms.

Number of interactions: 2564


## Run Heat Propagation
<a id='heat'></a>

In [4]:
G_heat = nx.Graph(G_DEG)
seed_nodes = [n for n in DEG_list if n in G_DEG]
Wprime = visualizations.normalized_adj_matrix(G_DEG)
visualizations.draw_heat_prop(G_heat,
              seed_nodes,
              Wprime = Wprime,
              num_nodes = 200,
              edge_width=2,
              edge_smooth_enabled=True,
              edge_smooth_type='bezier',
              node_size_multiplier=5,
              hover = False,
              hover_connected_edges = False,
              largest_connected_component = True,
              physics_enabled=True,
              node_font_size=40,
              graph_id = 1)  

## Clustering 
<a id='cluster'></a>

Parameter information:
- **G_DEG**: background network filtered by DEG list, output of load_STRING_to_digraph
- **DG_universe**: full background network, output of create_graph.load_STRING_to_digraph 
- **seed_nodes**: list of DEG's, output of create_graph.create_DEG_list
- **Wprime**: will calculate automatically of not specified, output of visualizations.normalized_adj_matrix
- **num_top_genes**: number of genes to display in the output graph
- **cluster_size_cut_off**: color clusters below this threshhold grey
- **remove_stray_nodes**: remove custers below the cluster size cut off
- **r**: increases spacing between clusters. recommended number between 0.5 and 4.0
- **x_offset**: modify if some clusters are overlapping. Extra helpful when x_offset != y_offset
- **y_offset**: modify if some clusters are overlapping. Extra helpful when x_offset != y_offset
- **node_spacing**: recommended number between 500 and 2000
- **node_size_multiplier**: as you scale node_spacing, scale this number. Recommended number between 5 and 25
- **physics_enabled**: Nodes will bounce aroound when you click and drag them. Only set to True when number of nodes is 200 or less
- **node_font_size**: as you scale node_spacing, scale this number. Recommended number between 20 and 50
- **graph_id**: Allows rendering of multiple graphs in one notebook. Just make sure each graph has a unique id. 

In [16]:
heat_and_cluster.draw_clustering(G_DEG, DG_universe, seed_nodes, 
                rad_positions = False,
                Wprime = Wprime,
                num_top_genes = 200,
                cluster_size_cut_off = 5,
                remove_stray_nodes = True,
                node_spacing = 700,
                node_size_multiplier = 12,
                physics_enabled = True,
                node_font_size = 45,
                graph_id = 2,
                node_shadow_x = 6
               )

In [14]:
heat_and_cluster.draw_clustering(G_DEG, DG_universe, seed_nodes, 
                Wprime = Wprime,
                num_top_genes = 200,
                cluster_size_cut_off = 5,
                remove_stray_nodes = False,
                r = 2.0,
                x_offset = 10,
                y_offset = 78,
                node_spacing = 700,
                node_size_multiplier = 12,
                physics_enabled = False,
                node_font_size = 45,
                graph_id = 3,
                node_shadow_x = 6
               )